First, we need some imports and utility functions for classifying objects from this dataset so that object types atleast loosly match up with object types from the ucs dataset so we can compare apples to apples.

The classify_orbit function acts as a translator. It uses the laws of physics (specifically Kepler's Third Law) to convert the time it takes an object to orbit Earth (Period) into an altitude category (Class).

### **The Logic of Classification: Translating Physics into Categories**

To analyze the global orbital population, we use custom logic to translate raw database values into meaningful physical and operational categories.

**1. `classify_orbit` (The Physics Translator)**
This function uses the laws of orbital mechanics to group objects based on their **Orbital Period** (the time required to circle the Earth):
* **LEO (< 128 min):** The high-traffic "City Center" of low-altitude satellites.
* **MEO (128 – 1400 min):** The "Open Highway" primarily used by GPS/Navigation constellations.
* **GEO (1400 – 1460 min):** The "Precise Lane" where satellites appear stationary over the equator.
* **High Elliptical (> 1460 min):** A catch-all for deep-space loops and "Graveyard Orbits."
* **Sanitization:** It explicitly identifies missing or physically impossible data (0 or negative periods) as **Unknown**.



**2. `categorize_object` (Operational Status Map)**
This determines what an object *is* by comparing the SATCAT registry against our enriched UCS dataset:
* **Active Satellite:** Objects verified in both datasets (The "Current Inventory").
* **Inactive Satellite:** Payloads that are no longer operational (The "Dead Weight").
* **Rocket Bodies & Debris:** Specifically isolates spent boosters and fragments to identify the highest risk-contributors to the Kessler Syndrome.

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 100)

ucs_data = pd.read_csv('../data/clean/ucs_cleaned.csv')
df_debris = pd.read_csv('../data/original/satcat.csv')

def classify_orbit(row):
    period = row['period_minutes']
    
    if pd.isnull(period) or period <= 0:
        return 'Unknown'
    elif period < 128:
        return 'LEO'
    elif 1400 <= period <= 1460:
        return 'GEO'
    elif 128 <= period < 1400:
        return 'MEO'
    elif period > 1460:
        return 'High Elliptical / Deep'
    else:
        return 'Unknown'
    
def categorize_object(row):
    if row['source'] == 'both':
        return 'Active Satellite'
    elif row['object_type'] == 'PAY':
        return 'Inactive Satellite'
    elif row['object_type'] == 'R/B':
        return 'Rocket Body'
    elif row['object_type'] == 'DEB':
        return 'Debris'
    else:
        return 'Unknown'

### **Standardizing the Global Orbital Catalog**

In this phase, we transform the raw SATCAT data into a structured "Master Dataset" by normalizing field names, filtering for active threats, and enriching the data with external mass profiles.

**Key Operations Performed:**
* **Column Normalization:** Renamed raw database headers (e.g., `NORAD_CAT_ID`, `PERIOD`) into human-readable, snake_case formats for consistent coding.
* **Active Threat Filtration:** Filtered the catalog to include only objects currently in orbit (where `decay_date` is null), as decayed objects no longer pose a collision risk.
* **Temporal Standardization:** Converted date strings into `datetime` objects and extracted `launch_year` and `decay_year` to support historical trend analysis.
* **Data Enrichment (The Merge):** Joined the SATCAT records with our cleaned UCS dataset. This attaches high-fidelity attributes like `purpose`, `users`, and our sophisticated `launch_mass_kg` estimations to the global catalog.
* **Categorization & Sanitization:** Applied physics-based logic to classify orbits and neutralized physically impossible `0.0 kg` mass values by converting them to `NaN`.

In [ ]:
debris_mapping = {
    'OBJECT_NAME': 'object_name',
    'OBJECT_ID': 'object_id',          
    'NORAD_CAT_ID': 'norad_id',        
    'OBJECT_TYPE': 'object_type',      
    'OPS_STATUS_CODE': 'ops_status',   
    'OWNER': 'owner',
    'LAUNCH_DATE': 'launch_date',
    'LAUNCH_SITE': 'launch_site',
    'DECAY_DATE': 'decay_date',
    'PERIOD': 'period_minutes',
    'INCLINATION': 'inclination_degrees',
    'APOGEE': 'apogee_km',
    'PERIGEE': 'perigee_km',
    'RCS': 'rcs',                      
    'DATA_STATUS_CODE': 'data_status', 
    'ORBIT_CENTER': 'orbit_center',    
    'ORBIT_TYPE': 'orbit_type'         
}

df_debris.rename(columns=debris_mapping, inplace=True)

current_junk = df_debris[df_debris['decay_date'].isnull()].copy()
current_junk['orbit_class'] = current_junk.apply(classify_orbit, axis=1)

merged_data = current_junk.merge(
    ucs_data[['norad_id', 'users', 'purpose', 'launch_mass_kg']], 
    on='norad_id', 
    how='left', 
    indicator='source'
)

# Convert strings to datetime objects
merged_data['launch_date'] = pd.to_datetime(merged_data['launch_date'], errors='coerce')
merged_data['decay_date'] = pd.to_datetime(merged_data['decay_date'], errors='coerce')

# Extract years for historical trend analysis
merged_data['launch_year'] = merged_data['launch_date'].dt.year
merged_data['decay_year'] = merged_data['decay_date'].dt.year

merged_data['category'] = merged_data.apply(categorize_object, axis=1)
merged_data['launch_mass_kg'] = merged_data['launch_mass_kg'].replace(0, np.nan)

### **Addressing the 'Mass Transparency Gap'**

During exploratory analysis, it was discovered that **82.8%** of the tracked objects in the SATCAT dataset effectively lack reliable mass data (assigned as `0.0 kg` in the raw catalog). Since a zero-mass physical object is an impossibility in orbital mechanics, these values represent missing data rather than actual measurements.

**Our Two-Tiered Correction Strategy:**
* **Tier 1: Active Payloads (Enriched):** We have integrated high-fidelity mass data from the UCS dataset. This covers the ~17% of objects identified as Active Satellites, using sophisticated median fills (grouped by Class and Purpose) performed during the UCS cleaning phase.
* **Tier 2: Debris & Fragments (Neutralized):** For the remaining **82.8%** of objects (Debris, Rocket Bodies, and Inactive Satellites not listed in the UCS registry), we are converting the impossible `0.0` values to `NaN`. This prevents "Zero-Mass" ghosts from skewing our statistical models.

**The Impact of this Cleaning Step:**
* **Statistical Accuracy:** Calculations like `.mean()` will now ignore missing values instead of being dragged down by zeros.
* **Visualization Integrity:** This eliminates the artificial "Outlier Spike" at the zero-line in histograms.
* **Future-Proofing:** This sets the stage for **RCS-Based Modeling**, where we will attempt to estimate the mass of these remaining `NaN` fragments based on their Radar Cross Section.

In [ ]:
if 'launch_mass_kg' in merged_data.columns:
    merged_data['launch_mass_kg'] = merged_data['launch_mass_kg'].replace(0, np.nan)

### **Data Validation: Analyzing Mass Coverage**

Now that the datasets are merged and physically impossible values have been neutralized, we perform a "Sanity Check" to verify the mass profile of our catalog. This snapshot confirms the effectiveness of our data enrichment and defines the exact scale of the remaining **Mass Transparency Gap**.

In [ ]:
has_mass = merged_data[merged_data['launch_mass_kg'].notnull()].copy()

total_objects = len(merged_data)
known_mass_count = len(has_mass)
gap_count = total_objects - known_mass_count

print(f"--- Dataset Mass Profile ---")
print(f"Total Objects in Catalog: {total_objects:,}")
print(f"Objects with Valid Mass:  {known_mass_count:,} ({known_mass_count/total_objects:.1%})")
print(f"Mass Transparency Gap:    {gap_count:,} ({gap_count/total_objects:.1%})")

has_mass[['object_name', 'category', 'orbit_class', 'launch_mass_kg']].head(10)

### **Tier 1: Synthetic Mass Averaging (Imputation)**

**The Problem:**
Our initial audit revealed a **Mass Transparency Gap** of **82.8%**. The vast majority of debris and rocket bodies have no public mass data (`NaN`), which makes calculating the total "Kinetic Fuel" of the orbital environment impossible with raw data alone.

**The Solution:**
To create a baseline for our EDA models, we will implement a **Synthetic Mass Fill**. We create a new column, `proxy_mass_kg`, which preserves the high-fidelity UCS data where available, but fills the gaps with **Conservative Categorical Averages** based on European Space Agency (ESA) debris reports.

**Imputation Logic:**
* **Rocket Bodies:** Assigned **2,000 kg** (Conservative average for upper stages like SL-12, Centaur, Falcon 9).
* **Inactive Satellites:** Assigned **1,000 kg** (Average bus mass for historical payloads).
* **Debris:** Assigned **0.1 kg** (Statistical average for trackable fragments >10cm).

In [ ]:
merged_data['proxy_mass_kg'] = merged_data['launch_mass_kg']

# define categorical averages (Conservative estimates based on historical bus sizes)
# Source: European Space Agency (ESA) Annual Space Environment Report averages
mass_proxies = {
    'Rocket Body': 2000.0,        # Average upper stage mass (conservative)
    'Inactive Satellite': 1000.0, # Average historical payload
    'Debris': 0.1                 # Significant debris fragments
}

# apply the fill only where mass is missing
for category, mass_val in mass_proxies.items():
    mask = (merged_data['category'] == category) & (merged_data['proxy_mass_kg'].isna())
    merged_data.loc[mask, 'proxy_mass_kg'] = mass_val

old_coverage = merged_data['launch_mass_kg'].notnull().mean()
new_coverage = merged_data['proxy_mass_kg'].notnull().mean()

print(f"--- Mass Data Upgrade ---")
print(f"Original Coverage: {old_coverage:.1%}")
print(f"Synthetic Coverage: {new_coverage:.1%}")

### **Final Dataset Audit: Global Orbital Composition**

Before exporting the master dataset, we perform a final audit of the orbital population. This summary confirms that every object currently in orbit has been successfully categorized by both its **Functional Status** (Active, Inactive, Debris) and its **Physical Location** (LEO, MEO, GEO).

In [ ]:
print("--- Dataset Mass Profile ---")
total_objects = len(merged_data)

# 1. Audit the Original Data (The "Real" Mass)
real_mass_count = merged_data['launch_mass_kg'].notnull().sum()
print(f"Total Objects in Catalog: {total_objects:,}")
print(f"Objects with Valid Real Mass:  {real_mass_count:,} ({real_mass_count/total_objects:.1%})")

# 2. Audit the Tier 1 Proxy Data (The "Modeled" Mass)
proxy_mass_count = merged_data['proxy_mass_kg'].notnull().sum()
print(f"Objects with Tier 1 Proxy Mass: {proxy_mass_count:,} ({proxy_mass_count/total_objects:.1%})")

# 3. Calculate the Transparency Gap
missing_real_mass = total_objects - real_mass_count
print(f"Mass Transparency Gap (Raw):    {missing_real_mass:,} ({missing_real_mass/total_objects:.1%})")

print("\n--- Composition of the Skies (Functional Status) ---")
print(merged_data['category'].value_counts())

print("\n--- Orbital Distribution (Physical Location) ---")
print(merged_data['orbit_class'].value_counts())

In [ ]:
merged_data.to_csv('../data/clean/orbital_clutter_cleaned.csv', index=False)

print("File saved successfully!")

Future considerations: Consider finding orbital data for astroids and other objects not originating from earth (not man made but naturally occurring).
This includes things like astroids, comets, and other 'space rocks' that could be hazardous to satellites orbiting at high speeds.